<a href="https://www.kaggle.com/code/gourabr0y555/ps-s4e3-catboost-eda-importance-plots?scriptVersionId=168942602" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="border-radius: 15px;
            border: 2px solid red;
            color:white;
            font-family: Helvetica;
            text-align: center;
            padding: 20px;
            width: 800px;
            text-align: center;
            background-color: #FF7F7F;
            font-size: 44px;
            text-shadow: 2px 2px 4px #000;">
    Short & Easy Notebook </br>
    to get </br>
    Started
</div>


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div style="border-radius: 15px;
            border: 2px solid blue;
            color:white;
            font-family: Helvetica;
            text-align: center;
            padding: 20px;
            width: 800px;
            text-align: center;
            background-color: #ADD8E6;
            font-size: 44px;
            text-shadow: 2px 2px 4px #000;">
    Load Data
</div>


In [ ]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')

In [ ]:
train_data2 = pd.read_csv('/kaggle/input/steel-plates-faults/SteelPlatesFaults.csv')

In [ ]:
train_data = pd.concat([train_data, train_data2], ignore_index=True)
train_data.reset_index(inplace=True, drop=True)

In [ ]:
train_data.head().style.background_gradient()

<div style="border-radius: 15px;
            border: 2px solid blue;
            color:white;
            font-family: Helvetica;
            text-align: center;
            padding: 20px;
            width: 800px;
            text-align: center;
            background-color: #ADD8E6;
            font-size: 44px;
            text-shadow: 2px 2px 4px #000;">
    Remove Duplicates & replace null values
</div>

In [ ]:
train_data.duplicated().sum()

In [ ]:
test_data.duplicated().sum()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum().sum()

#### No duplicates & no null values [missing index values can be ignored]

#### We don't need to insert any values for now. If needed we could have used SimpleImputer from sklearn.impute library

<div style="border-radius: 15px;
            border: 2px solid blue;
            color:white;
            font-family: Helvetica;
            text-align: center;
            padding: 20px;
            width: 800px;
            text-align: center;
            background-color: #ADD8E6;
            font-size: 44px;
            text-shadow: 2px 2px 4px #000;">
    Feature Engineering
</div>

In [ ]:
for col in train_data.columns:
    print(col)

In [ ]:
train_data['X_range'] = train_data['X_Maximum'] - train_data['X_Minimum']
train_data['Y_range'] = train_data['Y_Maximum'] - train_data['Y_Minimum']
train_data['Luminosity_range'] = train_data['Maximum_of_Luminosity'] - train_data['Minimum_of_Luminosity']
train_data['Area'] = np.exp(train_data['LogOfAreas'])
train_data['X_Index'] = np.exp(train_data['Log_X_Index'])
train_data['Y_Index'] = np.exp(train_data['Log_Y_Index'])

test_data['X_range'] = test_data['X_Maximum'] - test_data['X_Minimum']
test_data['Y_range'] = test_data['Y_Maximum'] - test_data['Y_Minimum']
test_data['Luminosity_range'] = test_data['Maximum_of_Luminosity'] - test_data['Minimum_of_Luminosity']
test_data['Area'] = np.exp(test_data['LogOfAreas'])
test_data['X_Index'] = np.exp(test_data['Log_X_Index'])
test_data['Y_Index'] = np.exp(test_data['Log_Y_Index'])

In [ ]:
for col in train_data.columns:
    print(col)

<div style="border-radius: 15px;
            border: 2px solid blue;
            color:white;
            font-family: Helvetica;
            text-align: center;
            padding: 20px;
            width: 800px;
            text-align: center;
            background-color: #ADD8E6;
            font-size: 44px;
            text-shadow: 2px 2px 4px #000;">
    Heatmap to show correlation
</div>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
corr = train_data.corr()

plt.figure(figsize = (40, 40), dpi = 100)
sns.heatmap(corr, cmap = 'YlGnBu', fmt = '.2f', annot = True)
plt.title('Correlation heatmap')
plt.show()

<div style="border-radius: 15px;
            border: 2px solid blue;
            color:white;
            font-family: Helvetica;
            text-align: center;
            padding: 20px;
            width: 800px;
            text-align: center;
            background-color: #ADD8E6;
            font-size: 44px;
            text-shadow: 2px 2px 4px #000;">
    Preprocessing
</div>


In [ ]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [ ]:
preprocess = ColumnTransformer([
    ('OneHot', OneHotEncoder(handle_unknown = 'ignore'), make_column_selector(dtype_include = object)),
    ('Scale', StandardScaler(), make_column_selector(dtype_include = np.number))
])

In [ ]:
features = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']

In [ ]:
Y_train = train_data[features]
train_data.drop(features, axis = 1, inplace = True)

In [ ]:
X_train = train_data.drop('id', axis = 1)
X_test = test_data.drop('id', axis = 1)

In [ ]:
preprocess.fit(pd.concat([X_train, X_test]))
X_cols = X_train.columns

X_train = preprocess.transform(X_train)
X_test = preprocess.transform(X_test)

In [ ]:
X_train = pd.DataFrame(X_train)
X_train.columns = X_cols

<div style="border-radius: 15px;
            border: 2px solid blue;
            color:white;
            font-family: Helvetica;
            text-align: center;
            padding: 20px;
            width: 800px;
            text-align: center;
            background-color: #ADD8E6;
            font-size: 44px;
            text-shadow: 2px 2px 4px #000;">
    Model training & predictions
</div>

In [ ]:
import catboost
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 600,  
    "learning_rate": 0.05,  
    "l2_leaf_reg": 3,  
    "bagging_temperature": 0.5,  
    "random_strength": 0.1,  
    "border_count": 128,  
    "thread_count": 4,  
    "verbose": False,  
    "random_state": 42
}

model = catboost.CatBoostClassifier(**params)

In [ ]:
X_train1, X_test1, Y_train1, Y_test = train_test_split(X_train, Y_train, test_size = 0.3, random_state = 42)

result = pd.DataFrame()

for feature in features:
    model.fit(X_train1, Y_train1[feature])
    preds_test = model.predict_proba(X_test1)[:, 1]
    print(f'AUC Score for {feature}: {roc_auc_score(Y_test[feature], preds_test)}\n')
    preds = model.predict_proba(X_test)[:, 1]
    result[feature] = preds

In [ ]:
submissions = pd.DataFrame({'id': test_data.id, 'Pastry': result.Pastry, 'Z_Scratch': result.Z_Scratch, 'K_Scatch': result.K_Scatch, 'Stains': result.Stains, 'Dirtiness': result.Dirtiness,'Bumps': result.Bumps,'Other_Faults': result.Other_Faults})

submissions.to_csv('submission.csv', index = False)

In [ ]:
submissions.head().style.background_gradient()